In [ ]:
import math

In [ ]:
def prob_model(query, documents, R, top_p_stems):
    # Initialize probabilities
    PofR = 0.5
    PofNotR = 0.5
    Poft_R = {}
    Poft_NotR = {}
    for term in query:
        Poft_R[term] = 0.5
        Poft_NotR[term] = documents.count(term) / len(documents)


    
    
    # Update probabilities based on relevant documents
    for doc in R:
        for term in query:
            if term in doc:
                Poft_R[term] += 1 / len(R)
            else:
                Poft_NotR[term] += 1 / (len(documents) - len(R))
        PofR += 1 / len(R)
        PofNotR += 1 / (len(documents) - len(R))
    
    # Normalize probabilities
    PofR /= len(documents)
    PofNotR /= len(documents)
    for term in query:
        Poft_R[term] /= len(R)
        Poft_NotR[term] /= (len(documents) - len(R))
    
    # Calculate scores
    scores = []
    for i, doc in enumerate(documents):
        score = 0
        for term in query:
            if term in top_p_stems[i]:
                w_iq = 1
            else:
                w_iq = 0
            if term in doc:
                w_ij = doc.count(term)
            else:
                w_ij = 0
            score += w_iq * w_ij * (math.log(Poft_R[term] / (1 - Poft_R[term])) + math.log((1 - Poft_NotR[term]) / Poft_NotR[term]))
        scores.append((i, score))
    ranked_docs = [doc for doc, score in sorted(scores, key=lambda x: x[1], reverse=True)]
    return ranked_docs